# RNN with OneHot encoding
Combine two previous achievements.
Hold off on the stratified split.

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
tf.keras.backend.set_floatx('float64')


In [2]:
def load_fasta(filename):
    # assume file was preprocessed to contain one line per seq
    MIN_SEQ_LEN=200
    MAX_SEQ_LEN=25000
    DEFLINE='>'
    seqs=[]
    with open (filename,'r') as infile:
        for line in infile:
            if line[0]!=DEFLINE and len(line)>=MIN_SEQ_LEN and len(line)<=MAX_SEQ_LEN:
                line=line.rstrip()
                chars=np.array(list(line))
                seqs.append(chars.reshape(-1, 1)) # reshaped changes (any,) to (any,1)
    # no need for this -- should use array or nparray consistently
    nparray=np.array(seqs)
    return nparray

ncfile='ncRNA.fasta' 
print("Load "+ncfile)
nc_seqs = load_fasta(ncfile)
pcfile='pcRNA.fasta' 
print("Load "+pcfile)
pc_seqs = load_fasta(pcfile)

Load ncRNA.fasta
Load pcRNA.fasta


In [3]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
print("Fit")
encoder.fit(nc_seqs[0])
encoder.categories_

Fit


[array(['A', 'C', 'G', 'T'], dtype='<U1')]

In [4]:
print("non-coding")
nc_all=[]
for seq in nc_seqs:
    encoded=encoder.transform(seq)
    nc_all.append(encoded)
print("protein-coding")
pc_all=[]
for seq in pc_seqs:
    encoded=encoder.transform(seq)
    pc_all.append(encoded)

nc_all=np.array(nc_all).reshape(-1,1)
pc_all=np.array(pc_all).reshape(-1,1)
nc_all.shape,pc_all.shape

non-coding
protein-coding


((17711, 1), (20152, 1))

In [5]:
seq_len=None  # none indicates variable length
input_features=4  # one hot encoding of DNA means 4 categories
mlp2 = keras.models.Sequential([
    keras.layers.SimpleRNN(4, return_sequences=True, input_shape=[seq_len,input_features]),
    keras.layers.SimpleRNN(1),
])
bc=tf.keras.losses.BinaryCrossentropy(from_logits=False)
mlp2.compile(loss=bc, optimizer="Adam",metrics=["accuracy"])
history = mlp2.fit(X_train,y_train,epochs=5,validation_data=(X_valid,y_valid))

NameError: name 'X_train' is not defined

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()